### Contexte

In [36]:
import pandas as pd
import torch as t
import numpy as np

df =  pd.read_csv('data/WorldBankGroup_Statement_Of_Loans_and_Guarantees.csv')


C:\Users\matts\AppData\Local\Temp\ipykernel_4896\3107689279.py:5: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('data/WorldBankGroup_Statement_Of_Loans_and_Guarantees.csv')


### Data Preproccesing 

On supprime les colonnes:
- `"Currency of Commitment"` vide puisqu'on sait que toutes les valeurs monétaires sont en USD
- `"Country / Economy Code"` puisqu'on a aussi une colonne avec le noms complets des pays
- `"Guarantor Country / Economy Code"` Idem
- `"Project ID"` Les identifiants des projets ne nous intéressent pas

In [37]:
df.drop(columns=["Currency of Commitment"], inplace=True)
df.drop(columns=["Country / Economy Code"], inplace=True)
df.drop(columns=["Guarantor Country / Economy Code"], inplace=True)
df.drop(columns=["Project ID"], inplace=True)

Suppression des lignes pour lesquelles on trouve des NA dans les colonnes suivantes:
- `"Interest Rate"`: Taux de NA: 2.24% (Si on ne connait pas le taux d'interet, il est difficile de faire quoi que ce soit) 

In [39]:
df.dropna(subset=["Interest Rate"], inplace=True)

On précise quand on ne connait pas le projet que finance le pret et on définit une catégorie `"Other"`

In [44]:
df["Project Name"].fillna("Other", inplace=True)

----------------------------------------

In [40]:
df.shape

(1384312, 29)

In [30]:
df.columns

Index(['End of Period', 'Loan Number', 'Region', 'Country / Economy Code',
       'Country / Economy', 'Borrower', 'Guarantor Country / Economy Code',
       'Guarantor', 'Loan Type', 'Loan Status', 'Interest Rate', 'Project ID',
       'Project Name', 'Original Principal Amount (US$)',
       'Cancelled Amount (US$)', 'Undisbursed Amount (US$)',
       'Disbursed Amount (US$)', 'Repaid to IBRD (US$)', 'Due to IBRD (US$)',
       'Exchange Adjustment (US$)', 'Borrower's Obligation (US$)',
       'Sold 3rd Party (US$)', 'Repaid 3rd Party (US$)', 'Due 3rd Party (US$)',
       'Loans Held (US$)', 'First Repayment Date', 'Last Repayment Date',
       'Agreement Signing Date', 'Board Approval Date',
       'Effective Date (Most Recent)', 'Closed Date (Most Recent)',
       'Last Disbursement Date'],
      dtype='object')

In [ ]:
df.describe()

,Interest Rate,Original Principal Amount (US$),Cancelled Amount (US$),Undisbursed Amount (US$),Disbursed Amount (US$),Repaid to IBRD (US$),Due to IBRD (US$),Exchange Adjustment (US$),Borrower's Obligation (US$),Sold 3rd Party (US$),Repaid 3rd Party (US$),Due 3rd Party (US$),Loans Held (US$)
count,1.384312e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1.416099e+06,1416099.0,1.416099e+06
mean,4.386095e+00,8.515593e+07,1.044060e+07,8.231778e+06,6.640113e+07,4.303031e+07,2.303149e+07,-4.181523e+05,2.261334e+07,4.412919e+05,4.412919e+05,0.0,3.126327e+07
std,3.379310e+00,1.567567e+08,4.711666e+07,5.177833e+07,1.377436e+08,9.260381e+07,1.013686e+08,1.167973e+07,9.773967e+07,3.854916e+06,3.854916e+06,0.0,1.182531e+08
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.226600e+08,0.000000e+00,0.000000e+00,-1.483090e+08,-1.100785e+09,-2.600000e-01,0.000000e+00,0.000000e+00,0.0,-1.483090e+08
25%,0.000000e+00,1.220000e+07,0.000000e+00,0.000000e+00,6.750714e+06,2.889000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00
50%,5.300000e+00,3.350000e+07,2.622700e+02,0.000000e+00,2.357800e+07,1.512776e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00
75%,7.250000e+00,1.000000e+08,3.804000e+06,0.000000e+00,6.985200e+07,4.518534e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00
max,2.825000e+01,3.750000e+09,1.995000e+09,3.379011e+09,3.153531e+09,3.000000e+09,2.923282e+09,5.065767e+08,2.842924e+09,2.499620e+08,2.499620e+08,0.0,3.750000e+09


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1384312 entries, 0 to 1416098
Data columns (total 29 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   End of Period                    1384312 non-null  object 
 1   Loan Number                      1384312 non-null  object 
 2   Region                           1384312 non-null  object 
 3   Country / Economy                1384312 non-null  object 
 4   Borrower                         1374686 non-null  object 
 5   Guarantor                        1314652 non-null  object 
 6   Loan Type                        1384312 non-null  object 
 7   Loan Status                      1384312 non-null  object 
 8   Interest Rate                    1384312 non-null  float64
 9   Project Name                     1384312 non-null  object 
 10  Original Principal Amount (US$)  1384312 non-null  float64
 11  Cancelled Amount (US$)           1384312 non-null  floa

In [ ]:
(1416099 - 1384312) / 1416099

0.022446876948574924

In [ ]:
df.columns[df.isna().any()]

Index(['Country / Economy Code', 'Borrower',
       'Guarantor Country / Economy Code', 'Guarantor', 'Interest Rate',
       'Project ID', 'Project Name', 'First Repayment Date',
       'Last Repayment Date', 'Agreement Signing Date', 'Board Approval Date',
       'Effective Date (Most Recent)', 'Closed Date (Most Recent)',
       'Last Disbursement Date'],
      dtype='object')

In [ ]:
df.groupby('Loan Status').count()

,End of Period,Loan Number,Region,Country / Economy Code,Country / Economy,Borrower,Guarantor Country / Economy Code,Guarantor,Loan Type,Interest Rate,...,Repaid 3rd Party (US$),Due 3rd Party (US$),Loans Held (US$),First Repayment Date,Last Repayment Date,Agreement Signing Date,Board Approval Date,Effective Date (Most Recent),Closed Date (Most Recent),Last Disbursement Date
Loan Status,,,,,,,,,,,,,,,,,,,,,
Approved,8066,8066,8066,8059,8066,8065,8058,7954,8066,7314,...,8066,8066,8066,7475,7475,19,8066,621,8066,0
Cancelled,12678,12678,12678,12678,12678,12480,11005,10575,12678,12075,...,12678,12678,12678,12678,12678,11229,12678,12678,12245,788
Disbursed,83405,83405,83405,83405,83405,83405,83193,79445,83405,78540,...,83405,83405,83405,83405,83405,83405,83405,83405,83405,79190
Disbursing,89185,89185,89185,89185,89185,89185,89151,86969,89185,84491,...,89185,89185,89185,89185,89185,89185,89185,89185,89185,89171
Disbursing&Repaying,14455,14455,14455,14455,14455,14455,14455,14328,14455,14328,...,14455,14455,14455,14455,14455,14455,14455,14455,14455,14452
Draft,1,1,1,1,1,0,0,0,1,1,...,1,1,1,0,0,0,0,0,1,0
Effective,5592,5592,5592,5592,5592,5592,5522,5404,5592,3637,...,5592,5592,5592,3620,3620,5592,5592,5592,5592,42
Fully Cancelled,21726,21726,21726,21726,21726,21414,19230,19052,21726,17257,...,21726,21726,21726,21299,21299,19438,21726,21617,20998,1248
Fully Disbursed,27600,27600,27600,27600,27600,27600,27600,27321,27600,27321,...,27600,27600,27600,27600,27600,27600,27600,27600,27600,27600


In [ ]:
df['Loan Status'].unique()

array(['Fully Repaid', 'Repaid', 'Cancelled', 'Fully Cancelled',
       'Disbursed', 'Repaying', 'Fully Transferred', 'Disbursing',
       'Terminated', 'Disbursing&Repaying', 'Effective',
       'Fully Disbursed', 'Draft', 'Approved', 'Signed', 'Negotiated'],
      dtype=object)

### Algo 1) reseaux de neurone

### Algo 2) clustering k-means

### Algo 3) clustering densite

### Algo 4) Regression Logistique Multiclasse

### Algo 5) Random forest 

## Evaluation des modeles  

### Matrice de confusion

### Accuracy (Exactitude)

### Precision, Recall et F1-Score

### AUC / ROC multi-classes